# Credit Card Fraud Detection BISWAJIT

## Import Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Load Dataset

In [2]:
credit_card_data = pd.read_csv("creditcard.csv")

In [3]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
credit_card_data.shape

(284807, 31)

## Exploratory Data Analysis

In [5]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

#### Distribution of Transactions

In [6]:
credit_card_data["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [7]:
legit = credit_card_data[credit_card_data["Class"] == 0]
fraud = credit_card_data[credit_card_data["Class"] == 1]

#### Dataset is Highly unbalanced
0 -> Legit transaction
1 -> Fraud transaction

In [8]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


#### Statistical Analysis

In [9]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [10]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [11]:
credit_card_data.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Under Sampling
Build a sample Dataset of similar legit and Fraud Transactions i.e 492 Each

In [12]:
legit_sample = legit.sample(n = 492)

In [15]:
new_dataset = pd.concat([legit_sample,fraud], axis = 0)

In [16]:
new_dataset["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [17]:
new_dataset.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95623.489837,-0.069597,-0.097116,-0.017353,0.007319,0.006768,-0.013990,0.057856,-0.017669,-0.015680,...,0.001344,0.008356,0.012972,0.004913,-0.001686,-0.020346,-0.009739,-0.007571,0.019443,99.593618
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Features(X) and Targets(Y)

In [18]:
X = new_dataset.drop(columns = ["Class"], axis = 1)
Y = new_dataset.Class

In [19]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
237302,149179.0,-0.583746,1.116505,1.706403,4.558139,0.728325,1.515921,0.400174,-0.065512,-1.037200,...,0.095806,0.093918,0.766406,-0.234199,0.654356,-0.115330,0.514116,-0.341922,-0.033066,22.46
279642,169005.0,2.136650,0.100929,-2.485745,-0.300638,1.102161,-0.858744,0.797265,-0.482373,-0.171598,...,-0.088453,0.152415,0.585679,-0.216350,-1.003920,0.589061,0.282098,-0.086021,-0.094566,19.11
196544,131564.0,-0.956595,0.431712,1.169007,-3.922871,0.186059,-0.443379,1.276654,-0.183275,1.411881,...,-0.096864,-0.373118,-0.930285,-0.252714,0.465296,0.827475,-1.342543,-0.035925,0.037932,80.00
261542,160066.0,-1.152762,-0.566488,1.231622,-2.907505,-1.437100,-0.290336,-0.122691,0.469573,-2.561718,...,-0.208689,0.019144,-0.017759,-0.007558,-0.040216,0.621570,-0.158164,-0.144489,-0.029558,144.00
144867,86446.0,-1.500281,0.257187,-0.312867,-0.018272,2.764601,-0.542169,1.101715,0.153244,-0.842665,...,0.153098,0.051414,-0.313053,-0.413001,-0.300510,1.432310,-0.447908,-0.063583,0.074448,65.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [20]:
Y

237302    0
279642    0
196544    0
261542    0
144867    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

### Split into Training & Testing Data

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y, random_state = 2)

In [47]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
114208,73380.0,0.716534,-1.083823,0.879423,0.446448,-1.341022,-0.264413,-0.227645,-0.082805,0.987560,...,0.524420,-0.064118,-0.446526,-0.160517,0.519482,0.084034,0.910598,-0.074115,0.061772,265.16
69458,53437.0,-0.315190,-0.554613,1.949042,-1.724854,-1.204667,-0.516325,-0.339657,-0.079445,-2.119668,...,-0.108066,-0.371987,-0.645198,0.160998,0.346009,-0.377535,-0.495882,0.151666,0.145226,50.99
112120,72519.0,-0.044000,1.272578,-0.056934,-0.314074,0.990821,-1.220761,1.209168,-0.317318,-0.599658,...,-0.091958,0.028802,0.175151,-0.074579,-0.184965,-0.827997,0.038305,0.190179,0.249177,0.76
116404,74262.0,-2.250535,2.365755,-2.955491,0.089791,-2.830745,-0.844462,-0.174062,-0.407138,0.174216,...,-0.486537,0.256560,-0.466245,0.291105,0.242567,-1.279094,-1.123534,-0.630977,0.326839,311.28
240222,150494.0,1.852889,1.069593,-1.776101,4.617410,0.770413,-0.400859,-0.040970,0.089510,-0.217705,...,-0.288392,-0.157869,-0.176244,0.027437,-0.468006,0.058063,0.148263,0.042278,0.040573,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201660,133976.0,-8.310026,7.466997,-4.478918,-3.108998,-0.806974,-2.072373,1.245117,0.228744,6.350465,...,4.346208,-0.962342,0.473986,0.095756,-0.041749,1.168669,-0.084738,1.276184,-0.585246,0.68
57470,47923.0,0.364377,1.443523,-2.220907,2.036985,-1.237055,-1.728161,-2.058582,0.358895,-1.393306,...,0.310980,0.402730,-0.132129,-0.032977,0.460861,0.560404,0.409366,0.539668,0.296918,0.76
41395,40662.0,-4.446847,-0.014793,-5.126307,6.945130,5.269255,-4.297177,-2.591242,0.342671,-3.880663,...,-0.108006,0.247913,-0.049586,-0.226017,-0.401236,0.856124,0.661272,0.492560,0.971834,1.00
230476,146344.0,-0.099724,2.795414,-6.423856,3.247513,-1.632290,-2.766665,-2.312223,0.961014,-1.896001,...,0.340898,0.647714,0.126576,0.203953,0.008495,-0.174501,0.575295,0.152876,-0.098173,94.82


In [48]:
Y_train

114208    0
69458     0
112120    0
116404    1
240222    1
         ..
201660    0
57470     1
41395     1
230476    1
8386      0
Name: Class, Length: 738, dtype: int64

In [49]:
X_test

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
151011,94364.0,-15.192064,10.432528,-19.629515,8.046075,-12.838167,-1.875859,-21.359738,-3.717850,-5.969782,...,1.657476,-3.474097,1.765446,1.701257,0.381587,-1.413417,-1.023078,-2.634761,-0.463931,1.00
163680,116148.0,-2.755399,2.805390,-1.924727,-0.660657,-1.285916,-1.442886,-0.587491,1.751230,0.216768,...,0.111938,-0.258037,-0.717359,0.327120,0.020441,-0.144399,0.142781,-0.005452,-0.145907,13.94
65495,51613.0,-0.633761,-0.567122,0.486741,-3.629539,1.470949,3.356552,-0.932684,0.965101,-2.359621,...,0.004455,-0.050395,0.099264,-0.336678,0.992055,0.698650,-0.082173,0.297597,0.121343,18.00
139166,83045.0,1.354834,0.348300,-0.271459,0.360492,0.349700,-0.261401,0.086985,-0.139211,0.061793,...,-0.006799,-0.374479,-1.051813,-0.049168,-1.044951,0.417758,0.179739,-0.022068,0.017604,1.98
42756,41233.0,-10.645800,5.918307,-11.671043,8.807369,-7.975501,-3.586806,-13.616797,6.428169,-7.368451,...,-0.046170,2.571970,0.206809,-1.667801,0.558419,-0.027898,0.354254,0.273329,-0.152908,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10912,18712.0,-0.307795,1.261001,1.397062,0.642785,0.024935,-0.756654,0.691483,-0.289938,0.689776,...,0.018205,-0.288552,-0.610308,0.074144,0.297733,-0.755417,0.035154,-0.015314,0.162348,24.99
42528,41138.0,-4.595617,5.083690,-7.581015,7.546033,-6.949165,-1.729185,-8.190192,2.714670,-7.083169,...,1.682160,2.248971,0.566844,0.033744,0.591783,0.334229,0.386801,2.163898,0.983104,340.11
112840,72824.0,-1.111495,-0.257575,2.250210,1.152671,0.432904,1.254126,-0.584163,-0.609682,1.014602,...,-0.510614,0.862913,0.927825,-0.343058,-0.256268,-0.600742,-0.180331,0.026762,-0.358335,45.03
100248,67462.0,-1.202977,0.103618,1.470970,2.327969,1.203038,0.148992,0.063252,0.253188,-1.084507,...,0.071712,-0.084422,-0.437814,0.066687,-0.548506,-0.254421,-0.095345,-0.071668,0.291526,37.83


In [50]:
Y_test

151011    1
163680    0
65495     0
139166    0
42756     1
         ..
10912     0
42528     1
112840    1
100248    0
228482    0
Name: Class, Length: 246, dtype: int64

## Model Training
Logistic Regression because Binary Classification

In [51]:
model = LogisticRegression()

In [52]:
model = model.fit(X_train, Y_train)

## Evaluation
### Accuracy Score

In [53]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

0.9444444444444444

In [54]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.9105691056910569

## Deploy
### Creating Pickle File

In [55]:
import pickle

In [57]:
pickle.dump(model, open("model.pkl", 'wb'))

### Creating Flask API

In [63]:
import numpy as np
from flask import Flask, request, jsonify, render_template

In [64]:
app = Flask(__name__)
calc = pickle.load(open("model.pkl", 'rb'))